In [1]:
import tensorflow as tf
import numpy as np

In [76]:
size = 4
max_seqlen = 2*(size*size-1)
n_input = 2 # x,y coordinate
n_hidden = 10

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, max_seqlen, n_input])
seqlen = tf.placeholder(tf.int32, [None])

# Unstack to get a list of 'max_seqlen' tensors of shape (batch_size, n_input)
x_seq = tf.unstack(x, max_seqlen, 1)

lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

outputs, _ = tf.contrib.rnn.static_rnn(lstm_cell, x_seq, dtype=tf.float32, sequence_length=seqlen)
outputs = tf.stack(outputs)
outputs = tf.transpose(outputs, [1, 0, 2])

#We only want the last output for dynamic sequences
batch_size = tf.shape(outputs)[0]
index = tf.range(0, batch_size) * max_seqlen + (seqlen - 1)
outputs2 = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

x2 = tf.placeholder(tf.float32, [None, 5])

outputs3 = tf.concat([outputs2, x2], axis=1)
print(outputs3.get_shape())

sess = tf.Session()
sess.run(tf.global_variables_initializer())

(?, 15)


In [80]:
X = np.random.random((2, max_seqlen, n_input))
S = np.array([2, 8])

In [81]:
O = sess.run(outputs, {x:X, seqlen:S})
O2 = sess.run(outputs2, {x:X, seqlen:S})

In [82]:
O

array([[[ -4.61242087e-02,  -2.23900229e-02,  -2.98864748e-02,
          -1.46510424e-02,  -6.48697764e-02,  -5.02067395e-02,
           7.56573454e-02,   2.39838641e-02,   3.93305272e-02,
           3.72234844e-02],
        [ -6.13924898e-02,  -3.11461966e-02,  -3.78199033e-02,
          -2.70791575e-02,  -1.04058176e-01,  -7.27963373e-02,
           1.04129136e-01,   4.68788855e-02,   6.33814856e-02,
           5.34819365e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
      

In [83]:
O2

array([[-0.06139249, -0.0311462 , -0.0378199 , -0.02707916, -0.10405818,
        -0.07279634,  0.10412914,  0.04687889,  0.06338149,  0.05348194],
       [-0.21307515, -0.0533691 , -0.11397161,  0.05870926, -0.28898498,
        -0.07304505,  0.22790647, -0.0004362 ,  0.10146099,  0.19327648]], dtype=float32)

In [73]:
O2.shape

(2, 10)

In [84]:
X2 = np.random.random((2, 5))

In [85]:
sess.run(outputs3, {x:X, seqlen:S, x2:X2})

array([[ -6.13924898e-02,  -3.11461966e-02,  -3.78199033e-02,
         -2.70791575e-02,  -1.04058176e-01,  -7.27963373e-02,
          1.04129136e-01,   4.68788855e-02,   6.33814856e-02,
          5.34819365e-02,   7.31625378e-01,   6.80643082e-01,
          9.91036773e-01,   1.62072226e-01,   4.71789718e-01],
       [ -2.13075146e-01,  -5.33690974e-02,  -1.13971606e-01,
          5.87092564e-02,  -2.88984984e-01,  -7.30450451e-02,
          2.27906466e-01,  -4.36203089e-04,   1.01460993e-01,
          1.93276480e-01,   2.72767283e-02,   7.19966054e-01,
          4.69629914e-01,   4.59680974e-01,   1.41933113e-01]], dtype=float32)